In [2]:
cd /home/novella/PycharmProjects/stock-price-trend-prediction-remote-magi/

/home/novella/PycharmProjects/stock-price-trend-prediction-remote-magi


In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.cluster import MeanShift, KMeans
import numpy as np
import pickle
import tqdm 

np.random.seed(0)
plt.rcParams["figure.figsize"] = (16,9)

In [3]:
print(os.getcwd())

SRC_PATH_NAME = "data/stock_picking_july_2021/"
TOTAL_STOCKS = "676_chosen_stocks.txt"
STOCKS_FNAME = "_data_dwn_48_332__{}_2021-07-01_2021-08-06_1_1"
DST_FNAME_DATA = SRC_PATH_NAME + "stocks_info.dat"

with open(SRC_PATH_NAME + TOTAL_STOCKS) as f:
	li_stocks = [l.strip() for l in f.readlines()]
    
li_stocks

/home/novella/PycharmProjects/stock-price-trend-prediction-remote-magi


['AAPL',
 'NVDA',
 'META',
 'LYFT',
 'COIN',
 'AFRM',
 'AGNC',
 'PARA',
 'ERIC',
 'BKR',
 'XP',
 'ZM',
 'AMAT',
 'ADI',
 'DLO',
 'RKLB',
 'TEAM',
 'AZN',
 'CRWD',
 'DLTR',
 'FANG',
 'TMUS',
 'SHC',
 'WOOF',
 'WMG',
 'ISEE',
 'PEP',
 'CSGP',
 'MTCH',
 'LESL',
 'CG',
 'SBRA',
 'PENN',
 'SSRM',
 'EXPD',
 'EA',
 'TROW',
 'INSM',
 'LBRDK',
 'ACIW',
 'SYNH',
 'INMD',
 'TWNK',
 'APLS',
 'ABMD',
 'SRPT',
 'HSIC',
 'NCNO',
 'WSC',
 'SBNY',
 'ARVN',
 'ILMN',
 'AMLX',
 'IAC',
 'SBAC',
 'PTC',
 'TW',
 'SIVB',
 'LBRDA',
 'GNTX',
 'KNBE',
 'IEP',
 'PDCE',
 'SWAV',
 'BL',
 'JBHT',
 'TSEM',
 'XENE',
 'ALHC',
 'ZG',
 'SPT',
 'OMCL',
 'BEAM',
 'TCBI',
 'NBIX',
 'ZLAB',
 'OZK',
 'TFSL',
 'RRR',
 'BRKR',
 'MRCY',
 'VERX',
 'OLED',
 'MIDD',
 'SEIC',
 'VSAT',
 'NXST',
 'LANC',
 'SSB',
 'TTEK',
 'PPBI',
 'AAWW',
 'IPGP',
 'CASY',
 'MANH',
 'GLPG',
 'NSIT',
 'TTEC',
 'CENT',
 'LSXMB',
 'ODFL',
 'NDAQ',
 'LEGN',
 'AUB',
 'VRTX',
 'CLBK',
 'LOGI',
 'WBA',
 'FTNT',
 'UHALB',
 'ZBRA',
 'GLBE',
 'JJSF',
 'AMGN',
 

In [4]:
def read_nasdaq_stocks(src_data=SRC_PATH_NAME, dst_data=DST_FNAME_DATA):
    
    if not os.path.exists(dst_data):

        out_df = None
        df_prices_volume = pd.DataFrame()
        for i, i_stock in enumerate(li_stocks):
            stock_path = src_data + STOCKS_FNAME.format(i_stock) + "/"
            
            if os.path.exists(stock_path):
                print("Processing stock", i, i_stock)
                stock_dataframe = pd.DataFrame()

                tra_days = os.listdir(stock_path)
                tra_days.sort()

                for f in tra_days:
                    date = str(f.split("_")[1])

                    if "2021-08-04" == date:
                        continue

                    with open(stock_path + f) as file:                    
                        new_df = pd.read_csv(file)
                        new_df.index = pd.to_datetime(date + " " + new_df.loc[:, "time"])
                        new_df = new_df.drop("time", axis=1)
                        stock_dataframe = pd.concat([stock_dataframe, new_df], axis=0)

                stock_dataframe.columns = [i_stock + "-" + i for i in list(stock_dataframe.columns)]
                df_prices_volume = stock_dataframe if df_prices_volume.shape[0] == 0 else df_prices_volume.join(stock_dataframe, how="left")
            
            else:
                print("> Ignoring", i_stock, "does not exist")

        with open(dst_data, "wb") as f:
            pickle.dump(df_prices_volume, f)
    else:
        print("Loading pickle...")
        with open(dst_data, "rb") as f:
            df_prices_volume = pickle.load(f)
    
    return df_prices_volume


In [5]:
df_prices_volume = read_nasdaq_stocks()   
df_prices_volume

Loading pickle...


,AAPL-ask_1,AAPL-ask_size_1,AAPL-bid_1,AAPL- bid_size_1,NVDA-ask_1,NVDA-ask_size_1,NVDA-bid_1,NVDA- bid_size_1,META-ask_1,META-ask_size_1,...,UAL-bid_1,UAL- bid_size_1,SYNA-ask_1,SYNA-ask_size_1,SYNA-bid_1,SYNA- bid_size_1,ORLY-ask_1,ORLY-ask_size_1,ORLY-bid_1,ORLY- bid_size_1
time,,,,,,,,,,,,,,,,,,,,,
2021-07-01 09:30:00,1366300,39,1365300,1599,8054400,50,8050000,405,156400.0,5.0,...,527500,20,1606500,100,1501700,100,5669300,6,5625000,1
2021-07-01 09:30:01,1366000,125,1365600,33,8065000,255,8059300,10,156400.0,5.0,...,527900,12802,1606500,100,1501700,100,5669400,5,5656000,1
2021-07-01 09:30:02,1366800,106,1366600,110,8064400,108,8060600,100,156400.0,5.0,...,528000,193,1555600,2,1550000,2,5669400,5,5650000,78
2021-07-01 09:30:03,1366900,2,1366800,100,8061200,174,8059300,10,156400.0,5.0,...,528700,192,1555600,2,1550000,2,5669400,5,5656000,11
2021-07-01 09:30:04,1367200,5323,1367100,2,8061900,25,8058000,284,156400.0,5.0,...,528700,12,1560000,1,1551300,2,5669400,5,5656000,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-06 15:59:55,1460400,388,1460300,901,2036300,418,2036000,1632,146600.0,1000.0,...,476000,4977,1704300,66,1703100,298,6036600,100,6031000,182
2021-08-06 15:59:56,1460700,375,1460600,100,2036400,300,2036000,1471,NaN,NaN,...,476100,3389,1704300,18,1703100,298,6036600,287,6030700,100
2021-08-06 15:59:57,1461000,41024,1460900,100,2036800,200,2036200,702,NaN,NaN,...,476100,3289,1704300,18,1703100,298,6034400,130,6032400,163


In [6]:
#day = int(60*60*6.5)
#day_id = 12
#daily_midprice.iloc[day_id*day:day*(day_id+1),:].loc[:,"NVDA"]

In [7]:
VOLUMES_COLS  = [i for i in list(df_prices_volume) if "size_1" in i]
PRICES_COLUMS = [i for i in list(df_prices_volume) if "ask_1" in i or "bid_1" in i]

df_prices_volume.loc[:, PRICES_COLUMS] = df_prices_volume.loc[:, PRICES_COLUMS].fillna(method="ffill")
df_prices_volume.loc[:, VOLUMES_COLS]  = df_prices_volume.loc[:, VOLUMES_COLS].fillna(0)

GRANULARITY = int(60*60*6.5)
df_prices_volume = df_prices_volume.iloc[::GRANULARITY, :]  
df_prices_volume

,AAPL-ask_1,AAPL-ask_size_1,AAPL-bid_1,AAPL- bid_size_1,NVDA-ask_1,NVDA-ask_size_1,NVDA-bid_1,NVDA- bid_size_1,META-ask_1,META-ask_size_1,...,UAL-bid_1,UAL- bid_size_1,SYNA-ask_1,SYNA-ask_size_1,SYNA-bid_1,SYNA- bid_size_1,ORLY-ask_1,ORLY-ask_size_1,ORLY-bid_1,ORLY- bid_size_1
time,,,,,,,,,,,,,,,,,,,,,
2021-07-01 09:30:00,1366300,39,1365300,1599,8054400,50,8050000,405,156400.0,5.0,...,527500,20,1606500,100,1501700,100,5669300,6,5625000,1
2021-07-02 09:30:00,1379400,36,1379000,100,8183300,6,8172600,3,155100.0,1000.0,...,531000,2000,1580000,1,1527200,100,5830000,1,5767200,82
2021-07-06 09:30:00,1401500,67,1400700,1,8299000,400,8295500,6,155500.0,1000.0,...,526700,24,1595000,100,1570000,1,5850000,50,5818800,1
2021-07-07 09:30:00,1436000,118,1435100,47,8344500,6,8339900,6,151500.0,30.0,...,509800,9,1592700,3,1558100,100,5978700,100,5807700,16
2021-07-08 09:30:00,1415700,213,1415600,221,7944100,6,7935000,32,148300.0,1000.0,...,490000,1,1503000,103,1470400,10,5853800,1,5794900,262
2021-07-09 09:30:00,1427700,65,1427100,421,7991900,14,7986700,5,149100.0,99.0,...,501300,100,1576000,1,1480000,1,5908000,20,5835400,447
2021-07-12 09:30:00,1462300,25,1462100,2328,8099300,1,8095900,6,151200.0,1000.0,...,501200,5,1559500,15,1542500,2,5945400,20,5860800,20
2021-07-13 09:30:00,1440700,1,1440300,807,8159500,6,8155200,6,150500.0,1100.0,...,497400,600,1616400,100,1526100,100,5949900,20,5900100,3
2021-07-14 09:30:00,1481100,7,1480400,2,8144300,1,8138000,1,151000.0,25.0,...,489000,7,1592700,500,1528000,1,5960000,8,5847800,100


In [8]:
STOCKS_COLS = list({c.split("-")[0] for c in df_prices_volume.columns})
len(STOCKS_COLS)

635

In [9]:
df_prices_volume_augment = df_prices_volume[:]

for st in STOCKS_COLS:
    cname_ask_vol, cname_bid_vol = st + "-ask_size_1", st + "- bid_size_1"
    cname_ask, cname_bid = st + "-ask_1", st + "-bid_1"

    ask, bid = df_prices_volume.loc[:,cname_ask], df_prices_volume.loc[:,cname_bid] 
    df_prices_volume_augment[st+"-midprice"] = (ask + bid) / 2
    
    past_mid = df_prices_volume_augment.loc[:,st+"-midprice"]
    futu_mid = df_prices_volume_augment.loc[:,st+"-midprice"].shift(-1)
    df_prices_volume_augment[st+"-return"] = (futu_mid - past_mid) / past_mid
        
    ask_vol, bid_vol = df_prices_volume.loc[:,cname_ask_vol], df_prices_volume.loc[:,cname_bid_vol] 
    df_prices_volume_augment[st+"-outvol"] = ask_vol + bid_vol

df_prices_volume_augment = df_prices_volume_augment.iloc[:-1,:]

/tmp/ipykernel_2037203/2166180946.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_prices_volume_augment[st+"-midprice"] = (ask + bid) / 2
/tmp/ipykernel_2037203/2166180946.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prices_volume_augment[st+"-midprice"] = (ask + bid) / 2
/tmp/ipykernel_2037203/2166180946.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd

/tmp/ipykernel_2037203/2166180946.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_prices_volume_augment[st+"-midprice"] = (ask + bid) / 2
/tmp/ipykernel_2037203/2166180946.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prices_volume_augment[st+"-midprice"] = (ask + bid) / 2
/tmp/ipykernel_2037203/2166180946.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd

/tmp/ipykernel_2037203/2166180946.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_prices_volume_augment[st+"-outvol"] = ask_vol + bid_vol
/tmp/ipykernel_2037203/2166180946.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prices_volume_augment[st+"-outvol"] = ask_vol + bid_vol
/tmp/ipykernel_2037203/2166180946.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using p

/tmp/ipykernel_2037203/2166180946.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_prices_volume_augment[st+"-midprice"] = (ask + bid) / 2
/tmp/ipykernel_2037203/2166180946.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prices_volume_augment[st+"-midprice"] = (ask + bid) / 2
/tmp/ipykernel_2037203/2166180946.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd

/tmp/ipykernel_2037203/2166180946.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_prices_volume_augment[st+"-outvol"] = ask_vol + bid_vol
/tmp/ipykernel_2037203/2166180946.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prices_volume_augment[st+"-outvol"] = ask_vol + bid_vol
/tmp/ipykernel_2037203/2166180946.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using p

/tmp/ipykernel_2037203/2166180946.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_prices_volume_augment[st+"-return"] = (futu_mid - past_mid) / past_mid
/tmp/ipykernel_2037203/2166180946.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prices_volume_augment[st+"-return"] = (futu_mid - past_mid) / past_mid
/tmp/ipykernel_2037203/2166180946.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider join

/tmp/ipykernel_2037203/2166180946.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_prices_volume_augment[st+"-midprice"] = (ask + bid) / 2
/tmp/ipykernel_2037203/2166180946.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prices_volume_augment[st+"-midprice"] = (ask + bid) / 2
/tmp/ipykernel_2037203/2166180946.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd

In [11]:
df_prices_volume_augment = df_prices_volume_augment.dropna(axis=1)
df_prices_volume_augment

,AAPL-ask_1,AAPL-ask_size_1,AAPL-bid_1,AAPL- bid_size_1,NVDA-ask_1,NVDA-ask_size_1,NVDA-bid_1,NVDA- bid_size_1,META-ask_1,META-ask_size_1,...,PACB-outvol,AXON-midprice,AXON-return,AXON-outvol,CWST-midprice,CWST-return,CWST-outvol,HZNP-midprice,HZNP-return,HZNP-outvol
time,,,,,,,,,,,,,,,,,,,,,
2021-07-01 09:30:00,1366300,39,1365300,1599,8054400,50,8050000,405,156400.0,5.0,...,410,1775000.0,-0.016169,304,640300.0,0.000000,212,950000.0,0.019105,2
2021-07-02 09:30:00,1379400,36,1379000,100,8183300,6,8172600,3,155100.0,1000.0,...,30,1746300.0,0.013572,13,640300.0,-0.005544,12,968150.0,-0.005526,400
2021-07-06 09:30:00,1401500,67,1400700,1,8299000,400,8295500,6,155500.0,1000.0,...,14,1770000.0,0.007599,40,636750.0,0.005575,120,962800.0,0.005193,210
2021-07-07 09:30:00,1436000,118,1435100,47,8344500,6,8339900,6,151500.0,30.0,...,541,1783450.0,-0.050688,7,640300.0,-0.011713,12,967800.0,-0.047117,400
2021-07-08 09:30:00,1415700,213,1415600,221,7944100,6,7935000,32,148300.0,1000.0,...,98,1693050.0,0.049526,46,632800.0,0.011852,210,922200.0,0.029657,219
2021-07-09 09:30:00,1427700,65,1427100,421,7991900,14,7986700,5,149100.0,99.0,...,55,1776900.0,0.004924,131,640300.0,0.002967,12,949550.0,0.001580,300
2021-07-12 09:30:00,1462300,25,1462100,2328,8099300,1,8095900,6,151200.0,1000.0,...,9,1785650.0,-0.007364,7,642200.0,0.002491,5,951050.0,0.005415,103
2021-07-13 09:30:00,1440700,1,1440300,807,8159500,6,8155200,6,150500.0,1100.0,...,360,1772500.0,0.034556,29,643800.0,-0.001398,125,956200.0,-0.006432,400
2021-07-14 09:30:00,1481100,7,1480400,2,8144300,1,8138000,1,151000.0,25.0,...,155,1833750.0,-0.032120,2,642900.0,-0.011744,101,950050.0,0.004631,102


In [ ]:
# PERCF = .1  # removes returns that are too high or too low 

# drop_col = np.where(((np.sum((daily_returns_df < -PERCF)*1, axis=0)==0)*1).astype(bool))[0]
# daily_returns_df = daily_returns_df.iloc[:, drop_col]
# daily_volume_gran = daily_volume_gran.iloc[:, drop_col]

# drop_col = np.where(((np.sum((daily_returns_df > PERCF)*1, axis=0)==0)*1).astype(bool))[0]
# daily_returns_df = daily_returns_df.iloc[:, drop_col]
# daily_volume_gran = daily_volume_gran.iloc[:, drop_col]

# daily_returns_df
# daily_returns_df = daily_returns_df.iloc[:, (.1 > daily_returns_df)]
# daily_returns_df

In [ ]:
# WIN = 20
# stds_df = daily_returns_df.rolling(WIN).std()
# #avgs_df = daily_returns_df.rolling(WIN).mean()
# # ax = daily_returns_df.plot()
# # stds_df.plot(ax=ax)
# stds_df.iloc[:, 10:50].plot()

In [12]:
df_prices_volume_augment

,AAPL-ask_1,AAPL-ask_size_1,AAPL-bid_1,AAPL- bid_size_1,NVDA-ask_1,NVDA-ask_size_1,NVDA-bid_1,NVDA- bid_size_1,META-ask_1,META-ask_size_1,...,PACB-outvol,AXON-midprice,AXON-return,AXON-outvol,CWST-midprice,CWST-return,CWST-outvol,HZNP-midprice,HZNP-return,HZNP-outvol
time,,,,,,,,,,,,,,,,,,,,,
2021-07-01 09:30:00,1366300,39,1365300,1599,8054400,50,8050000,405,156400.0,5.0,...,410,1775000.0,-0.016169,304,640300.0,0.000000,212,950000.0,0.019105,2
2021-07-02 09:30:00,1379400,36,1379000,100,8183300,6,8172600,3,155100.0,1000.0,...,30,1746300.0,0.013572,13,640300.0,-0.005544,12,968150.0,-0.005526,400
2021-07-06 09:30:00,1401500,67,1400700,1,8299000,400,8295500,6,155500.0,1000.0,...,14,1770000.0,0.007599,40,636750.0,0.005575,120,962800.0,0.005193,210
2021-07-07 09:30:00,1436000,118,1435100,47,8344500,6,8339900,6,151500.0,30.0,...,541,1783450.0,-0.050688,7,640300.0,-0.011713,12,967800.0,-0.047117,400
2021-07-08 09:30:00,1415700,213,1415600,221,7944100,6,7935000,32,148300.0,1000.0,...,98,1693050.0,0.049526,46,632800.0,0.011852,210,922200.0,0.029657,219
2021-07-09 09:30:00,1427700,65,1427100,421,7991900,14,7986700,5,149100.0,99.0,...,55,1776900.0,0.004924,131,640300.0,0.002967,12,949550.0,0.001580,300
2021-07-12 09:30:00,1462300,25,1462100,2328,8099300,1,8095900,6,151200.0,1000.0,...,9,1785650.0,-0.007364,7,642200.0,0.002491,5,951050.0,0.005415,103
2021-07-13 09:30:00,1440700,1,1440300,807,8159500,6,8155200,6,150500.0,1100.0,...,360,1772500.0,0.034556,29,643800.0,-0.001398,125,956200.0,-0.006432,400
2021-07-14 09:30:00,1481100,7,1480400,2,8144300,1,8138000,1,151000.0,25.0,...,155,1833750.0,-0.032120,2,642900.0,-0.011744,101,950050.0,0.004631,102


In [13]:
STOCKS_COLS = list({c.split("-")[0] for c in df_prices_volume_augment.columns})
len(STOCKS_COLS)

635

In [12]:
# np.where(df_prices_volume_augment.isnull().values)
# df_prices_volume_augment.iloc[0:10, 1508]

In [14]:
STOCKS_MAT = []
FEAT_MAT = []

dataset_list = []
for ist, st in enumerate(STOCKS_COLS):
    uti_columns = [c for c in df_prices_volume_augment.columns if c.startswith(st + "-")]
    if len(uti_columns) == 7:
        STOCKS_MAT.append(st)
        uti_columns = sorted(uti_columns)
        FEAT_MAT = [i.split("-")[1] for i in uti_columns]
        arr = df_prices_volume_augment.loc[:,uti_columns].values
        dataset_list.append(arr)
        
FEAT_MAT_PAR = [i+"-avg" for i in FEAT_MAT]
FEAT_MAT_PAR += [i+"-std" for i in FEAT_MAT]
FEAT_MAT = FEAT_MAT_PAR[:]
print(FEAT_MAT)


[' bid_size_1-avg', 'ask_1-avg', 'ask_size_1-avg', 'bid_1-avg', 'midprice-avg', 'outvol-avg', 'return-avg', ' bid_size_1-std', 'ask_1-std', 'ask_size_1-std', 'bid_1-std', 'midprice-std', 'outvol-std', 'return-std']


In [15]:
dataset = np.array(dataset_list)  # STOCKS x 24 x 7
dataset_m = np.mean(dataset, axis=1) # stocks x 7 
dataset_s = np.std(dataset, axis=1) # stocks x 7 
dataset_ms = np.hstack([dataset_m, dataset_s]) # stocks x 7 

df_final = pd.DataFrame(dataset_ms, columns=FEAT_MAT, index=STOCKS_MAT)
df_final["sharpe-r"] = df_final["return-avg"] / df_final["return-std"]
df_final

,bid_size_1-avg,ask_1-avg,ask_size_1-avg,bid_1-avg,midprice-avg,outvol-avg,return-avg,bid_size_1-std,ask_1-std,ask_size_1-std,bid_1-std,midprice-std,outvol-std,return-std,sharpe-r
VICR,77.958333,1.127454e+06,73.875000,1.036333e+06,1.081894e+06,151.833333,0.007151,47.447760,65019.484126,92.130936,77468.330447,64159.859454,128.062051,0.045554,0.156987
CCOI,116.541667,7.818042e+05,66.458333,7.560667e+05,7.689354e+05,183.000000,-0.000871,83.950868,13662.707125,89.465533,13190.674315,10186.975745,129.418250,0.015404,-0.056550
HPK,57.166667,1.151083e+05,78.208333,1.009083e+05,1.080083e+05,135.375000,0.002185,66.298358,5749.776566,64.206035,6929.340798,4831.249551,95.096798,0.055482,0.039391
CTSH,466.375000,7.011458e+05,740.708333,6.943375e+05,6.977417e+05,1207.083333,0.002442,1102.048956,20823.904116,1681.194999,20493.816395,20567.544552,1871.289482,0.016997,0.143697
PCVX,94.833333,2.313542e+05,106.625000,2.105375e+05,2.209458e+05,201.458333,0.002552,133.836239,10312.693601,135.714717,11475.220132,10068.628224,197.091556,0.064048,0.039846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CPRT,201.166667,1.427846e+06,147.375000,1.403212e+06,1.415529e+06,348.541667,0.004092,268.294097,45281.655402,250.126770,47209.306220,45545.204003,325.598421,0.015020,0.272442
PACB,251.000000,3.111250e+05,403.083333,3.067208e+05,3.089229e+05,654.083333,0.000611,500.736291,22047.415759,471.414619,22427.326694,22191.937252,667.172761,0.048663,0.012549
AXON,112.083333,1.822429e+06,44.250000,1.782512e+06,1.802471e+06,156.333333,0.004823,169.002938,55059.524450,70.599959,55131.753649,54112.960394,164.943846,0.022608,0.213312
CWST,50.416667,6.676083e+05,53.000000,6.315625e+05,6.495854e+05,103.416667,0.002455,60.872761,17386.367185,84.759955,17879.150159,15934.347591,79.104844,0.020482,0.119863


In [16]:
m = df_final.mean()
s = df_final.std()
df_final_norm = (df_final-m)/ s

df_final_norm

df_final_norm = df_final_norm.drop([' bid_size_1-avg', 'ask_1-avg', 'ask_size_1-avg', 'bid_1-avg', ' bid_size_1-std', 'ask_1-std', 'ask_size_1-std', 'bid_1-std'], axis=1)
df_final_norm

,midprice-avg,outvol-avg,return-avg,midprice-std,outvol-std,return-std,sharpe-r
VICR,-0.092511,-0.367639,1.043118,0.077569,-0.426516,0.536958,0.942505
CCOI,-0.214823,-0.337499,-0.149687,-0.284436,-0.425475,-0.571575,-0.445641
HPK,-0.473130,-0.383556,0.304769,-0.320358,-0.451816,0.901980,0.178043
CTSH,-0.242647,0.652849,0.342976,-0.214812,0.911373,-0.512986,0.856110
PCVX,-0.428991,-0.319649,0.359272,-0.285230,-0.373537,1.216917,0.181002
...,...,...,...,...,...,...,...
CPRT,0.037882,-0.177411,0.588239,-0.047283,-0.274911,-0.585687,1.693042
PACB,-0.394607,0.118066,0.070622,-0.203917,-0.012760,0.651253,0.003552
AXON,0.189108,-0.363288,0.696870,0.010183,-0.398210,-0.306682,1.308659
CWST,-0.261468,-0.414461,0.344845,-0.245888,-0.464089,-0.384869,0.701173


In [17]:
from openTSNE import TSNE  

tsne = TSNE(
    perplexity=30,
    metric="euclidean",
    n_jobs=8,
    random_state=42,
    verbose=True,
    n_components=3
)

In [18]:
embedding_train = tsne.fit(df_final_norm)

--------------------------------------------------------------------------------
TSNE(n_components=3, n_jobs=8, random_state=42, verbose=True)
--------------------------------------------------------------------------------
===> Finding 90 nearest neighbors using exact search using euclidean distance...
   --> Time elapsed: 0.02 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 0.02 seconds
===> Calculating PCA-based initialization...
   --> Time elapsed: 0.04 seconds
===> Running optimization with exaggeration=12.00, lr=200.00 for 250 iterations...
Iteration   50, KL divergence 2.6124, 50 iterations in 0.0732 sec
Iteration  100, KL divergence 2.5675, 50 iterations in 0.0697 sec
Iteration  150, KL divergence 2.5675, 50 iterations in 0.0702 sec
Iteration  200, KL divergence 2.5675, 50 iterations in 0.0745 sec
Iteration  250, KL divergence 2.5675, 50 iterations in 0.0590 sec
   --> Time elapsed: 0.35 seconds
===> Running optimization with exaggeration=1.00, lr=200.00 for 5

In [25]:
#import seaborn as sns
#palette = sns.color_palette("bright", 10)
#sns.scatterplot(embedding_train[:,0], embedding_train[:,1], legend='full', palette=palette)

# print(embedding_train)

K=3
clustering = KMeans(n_clusters=K).fit(embedding_train)
clustering_labels = clustering.labels_

fig = px.scatter_3d(x=embedding_train[:,0], y=embedding_train[:,1], z=embedding_train[:,2], color=clustering_labels, text=df_final_norm.index)
fig.show()

In [26]:
import umap 

reducer = umap.UMAP(random_state=42)
reducer.fit(df_final_norm)
embedding_train_umap = reducer.transform(df_final_norm)

K = 3
clustering = KMeans(n_clusters=K).fit(embedding_train)
cldf_final_normustering_labels = clustering.labels_

fig = px.scatter(x=embedding_train_umap[:,0], y=embedding_train_umap[:,1], text=df_final_norm.index, color=cldf_final_normustering_labels)
fig.show()

FINE

.

.

.

.

.

.

.

FINE 

In [17]:
risk_return = pd.DataFrame()
risk_return["return"] = daily_returns_df.mean(axis=0)
risk_return["risk"] = daily_returns_df.std(axis=0)
risk_return["avg-mid"] = daily_midprice_gran.mean(axis=0)
risk_return["srp-ratio"] = risk_return["return"] / risk_return["risk"]
risk_return["out-vol"] = daily_volume_gran.mean(axis=0)

# risk_return["symbol"] = daily_returns_df.columns
# risk_return = risk_return  metric="euclidean",
    n_jobs=8,
    random_state=42,
    verbose=True,
    n_components=3
).drop("ALHC")
risk_return = (risk_return -  risk_return.mean()) / risk_return.std()
risk_return

NameError: name 'daily_returns_df' is not defined

In [ ]:
vis_out = ["AMZN", "GOOG", "GOOGL", "BKNG", "MELI", "GRFS", "TRMD", "ERIC"]
risk_return = risk_return.drop(vis_out, axis=0)

In [ ]:
  metric="euclidean",
    n_jobs=8,
    random_state=42,
    verbose=True,
    n_components=3
)fig = px.scatter_3d(risk_return, x='avg-mid', y='srp-ratio', z='out-vol', text=risk_return.index)
fig.show()

In [ ]:
K = 10
cols = ['srp-ratio', 'out-vol']
risk_return = risk_return.dropna()[cols]
clustering = KMeans(n_clusters=K).fit(risk_return.values)
clustering_labels = clustering.labels_
risk_return["cluster"] = clustering_labels
risk_return["sym"] = risk_return.index
risk_return["type"] = ["normal"]*len(risk_return.index)

  metric="euclidean",
    n_jobs=8,
    random_state=42,
    verbose=True,
    n_components=3
)  metric="euclidean",
    n_jobs=8,
    random_state=42,
    verbose=True,
    n_components=3
)
risk_return

In [ ]:
  metric="euclidean",
    n_jobs=8,
    random_state=42,
    verbose=True,
    n_components=3
)li = []
print(K)
for k in range(K):
    rk1 = list(clustering.cluster_centers_[k]) + [K+2, "CLU{}".format(k), "centroid"]
    li.append(rk1)
   
cls = pd.DataFrame(li, columns=risk_return.columns)
cls 

risk_return_clu = risk_return[:]
risk_return_clu = pd.concat([risk_return_clu, cls])


fig = px.scatter_3d(risk_return_clu, x='srp-ratio', y='srp-ratio', z='out-vol', color='cluster', text="sym", symbol="type")
fig.show()

risk_return_clu

In [ ]:
  metric="euclidean",
    n_jobs=8,
    random_state=42,
    verbose=True,
    n_components=3
)def distance_epicenter(dfX, dfY, is_min):

    dfOut_list = []
    for index, row in dfY.iterrows():
        cluster = dfX[row["cluster"]]
        cluster_dims = row.values[:2]
        dist = np.linalg.norm(cluster_dims - cluster)
        dfOut_list.append(list(row.values) + [dist])

    dfOut = pd.DataFrame(dfOut_list)    
    dfOut.index = dfY.index
    dfOut.columns = list(dfY.columns) + ["epi-distance"]
    
    rows = []
    for i, clv in enumerate(dfX):
        row_clu = dfOut[dfOut["cluster"] == i]
        row_epi = row_clu["epi-distance"]
        row_id = row_epi.argmin() if is_min else row_epi.argmax()  
        rows.append(row_clu.iloc[row_id,:]["sym"])
    return rows

In [ ]:
d1 = distance_epicenter(clustering.cluster_centers_, risk_return, is_min=True)
d1

In [ ]:
  metric="euclidean",
    n_jobs=8,
    random_state=42,
    verbose=True,
    n_components=3
)d2 = distance_epicenter(clustering.cluster_centers_, risk_return, is_min=False)
d2

for s in d1:
    #risk_return_clu.at[s, "cluster"] = -4
    risk_return_clu.at[s, "type"] = "close"

    
for s in d2:
    #risk_return_clu.at[s, "cluster"] = -3
    risk_return_clu.at[s, "type"] = "far"
    
fig = px.scatter_3d(risk_return_clu, x='srp-ratio', y='srp-ratio', z='out-vol', color='cluster', text="sym", symbol="type")
fig.show()
fig.write_html("stocks-july.html")
risk_return_clu

In [ ]:
clustering.cluster_centers_

# NASDAQ Stocks with market cap from MEDIUM to MEGA

In [ ]:
features = ['Last Sale', 'Market Cap', 'Country', 'Volume', 'Sector']
NASDAQ_STOCKS = "nasdaq_medium_cap_stocks.csv"

df_csv = pd.read_csv(PATH_NAME + NASDAQ_STOCKS)

df_csv['Last Sale'] = df_csv['Last Sale'].str.replace('$', '', regex=True)
df_csv['Last Sale'] = pd.to_numeric(df_csv['Last Sale'])

df_csv  # FULL csv with 675 stocks in NASDAQ, having from MEDIUM to MEGA market cap

In [ ]:
# df_enc_cou = pd.get_dummies(df_csv['Country'])
df_enc_sec = pd.get_dummies(df_csv['Sector'])
df_encoded = pd.concat([df_csv['Volume'], df_csv['Last Sale']], axis=1) # df_enc_cou df_csv['Last Sale'],
df_encoded = df_encoded.set_index(df_csv["Symbol"])
df_encoded  # 675 stocks' last sale, market cap, volume, sector encoded for clustering

### Clustering stocks features
Clustering Last Sale, Market Cap, Volume and Sector

In [ ]:
K = 4
clustering = KMeans(n_clusters=K).fit(df_encoded.values)
clustering_labels = clustering.labels_
df_csv["Cluster"] = clustering_labels
df_csv

In [ ]:
fig = px.scatter(df_csv, x='Last Sale', y='Volume', color='Cluster', text="Symbol")
fig.write_html("stocks.html")
fig.show()

In [ ]:
#!open stocks.html

### Adding fetures for clustering

In [ ]:
df_encoded_2 = df_encoded.join(risk_return, how="inner")
df_encoded_2 = df_encoded_2.drop("risk", axis=1)
# df_encoded_2["Volume"] = np.log(df_encoded_2["Volume"])
# df_encoded_2["Last Sale"] = np.log(df_encoded_2["Last Sale"])
df_encoded_2 = df_encoded_2.drop(["PARA", "META", "AFRM"])
df_encoded_2

In [ ]:
K = 3
clustering = KMeans(n_clusters=K).fit(df_encoded_2.values)
clustering_labels = clustering.labels_
df_encoded_2_plot = df_encoded_2.copy(deep=True)
df_encoded_2_plot["Cluster"] = clustering_labels
df_encoded_2_plot["SYM"] = df_encoded_2_plot.index

fig = px.scatter_3d(df_encoded_2_plot, x='Volume', y='return', z="Last Sale", color='Cluster', text=df_encoded_2_plot.index)
fig.write_html("stocks.html")
fig.show()

In [ ]:
!open stocks.html

In [ ]:
df_encoded_2_plot

In [ ]:
from sklearn.metrics import pairwise_distances_argmin_min
closest, _ = pairwise_distances_argmin_min(clustering.cluster_centers_, df_encoded_2.values)
df_encoded_2.iloc[closest, :]

In [ ]:
def distance_epicenter(dfX, dfY, is_min):

    dfOut_list = []
    for index, row in dfY.iterrows():
        cluster = row["Cluster"]
        cluster_dims = row.values[:3]
        dist = np.linalg.norm(cluster_dims - cluster)
        dfOut_list.append(list(row.values) + [dist])

    dfOut = pd.DataFrame(dfOut_list)
    dfOut.index = dfY.index
    dfOut.columns = list(dfY.columns) + ["Epi-Distance"]
    dfOut = dfOut.groupby("Cluster")
    dfOut = dfOut.min() if is_min else dfOut.max()
    return dfOut

d1 = distance_epicenter(clustering.cluster_centers_, df_encoded_2_plot, is_min=True)
d2 = distance_epicenter(clustering.cluster_centers_, df_encoded_2_plot, is_min=False)
d1

In [ ]:
d2

In [ ]:
df_encoded_2

In [ ]:
# df_encoded_2_plot["Symbol"] =
fig = px.scatter_3d(df_encoded_2_plot, x='Last Sale', y='Market Cap', z='return', color='Cluster', text=df_encoded_2_plot.index)
fig.write_html("stocks_cluster_ret_risk.html")
fig.show()

In [ ]:
!open stocks_cluster_ret_risk.html

### Correlation Matrix

In [ ]:
df_encoded_2_corr = df_encoded_2.copy(deep=True)
df_encoded_2_corr = df_encoded_2_corr.drop(["Utilities", "Basic Materials"], axis=1)
df_cor = df_encoded_2_corr.corr()
df_cor

In [ ]:
plt.imshow(df_cor, cmap='hot', interpolation='none')
plt.colorbar()
plt.xticks(range(len(df_cor)), df_cor.columns, rotation=45)
plt.yticks(range(len(df_cor)), df_cor.columns)
plt.show()

In [ ]:
df_encoded_2_plot["Symbol"] = risk_return["Symbol"]
fig = px.scatter(df_encoded_2_plot, x='return', y='risk', text="Symbol", color="Cluster")
fig.show()